# Initial experiments Part 3

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from dataset import TUBerlinDataset, SketchyDataset
from utils import AbsoluteBezierPenPositionTokenizer
from prepare_data import stroke_to_bezier, convert_and_quantize_svg
from tqdm import tqdm
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

seed = 42
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)
    torch.cuda.empty_cache()

Using device: cuda


In [3]:
labels = ["cat"]
training_data0 = TUBerlinDataset(labels, download=True)
training_data1 = SketchyDataset(labels, download=True)
tokenizer = AbsoluteBezierPenPositionTokenizer(bins=32)

class BezierSketchDataset(Dataset):
    def __init__(
        self,
        svg_lists,
        tokenizer,
        max_len=400,
        cache_file="sketch_bezier_dataset.pkl",
    ):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_id = tokenizer.vocab["PAD"]
        max_num_tokens = 0

        # Try to load from cache
        try:
            with open(cache_file, "rb") as f:
                self.data = pickle.load(f)
            print(f"Loaded tokenized data from {cache_file}")
        except FileNotFoundError:
            for svg_list in svg_lists:
                for svg in tqdm(svg_list, desc="Tokenizing SVGs"):
                    svg = stroke_to_bezier(svg, num_samples=100, maxError=15.0)
                    q = convert_and_quantize_svg(svg, bins=256)
                    tokens = tokenizer.encode(q)

                    max_num_tokens = max(max_num_tokens, len(tokens))

                    # Truncate + Pad
                    tokens = tokens[:max_len]
                    tokens = tokens + [self.pad_id] * (max_len - len(tokens))
                    self.data.append(tokens)
                    
            print(f"Max number of tokens in a sequence: {max_num_tokens}")

            with open(cache_file, "wb") as f:
                pickle.dump(self.data, f)
            print(f"Saved tokenized data to {cache_file}")

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq[:-1])
        target_ids = torch.tensor(seq[1:])
        return input_ids, target_ids

    def __len__(self):
        return len(self.data)


dataset = BezierSketchDataset(
    svg_lists=[training_data0, training_data1],
    tokenizer=tokenizer,
)

Loading TUBerlin files: 100%|██████████| 1/1 [00:00<00:00, 90.45it/s]


Tokenizing SVGs: 100%|██████████| 508/508 [01:22<00:00,  6.15it/s]

Max number of tokens in a sequence: 920
Saved tokenized data to sketch_bezier_dataset.pkl


In [4]:
# from IPython.display import HTML, display

# svg_inline = ""
# for i in range(len(dataset)):
#     input_ids, target_ids, seq = dataset[i]
    
#     if len(seq) > 300:
#         svg = tokenizer.decode(seq)
#         svg_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Length: {len(seq)}</b><br>{svg}</div>'

# display(HTML(svg_inline))

In [5]:
def generate_square_subsequent_mask(sz: int):
    """Causal mask to stop attention to future positions"""
    return torch.triu(torch.ones(sz, sz), diagonal=1).bool()


class SketchTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=384, nhead=8, num_layers=6, max_len=400):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len

        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4 * d_model
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: (batch, seq_len) input tokens
        Returns: (batch, seq_len, vocab_size) logits
        """
        batch_size, seq_len = x.shape
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        x = self.embed(x) + self.pos_embed(positions)  # (batch, seq_len, d_model)
        x = x.transpose(0, 1)  # -> (seq_len, batch, d_model)
        mask = generate_square_subsequent_mask(seq_len).to(x.device)  # causal mask (seq_len, seq_len)
        x = self.transformer(x, mask=mask)  # (seq_len, batch, d_model)
        x = x.transpose(0, 1)  # back to (batch, seq_len, d_model)
        logits = self.fc_out(x)  # (batch, seq_len, vocab_size)
        return logits


def train_model(model, dataloader, vocab_size, epochs=10, lr=1e-4, device="cuda"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, target_ids in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            # Transformer expects shape (seq_len, batch, d_model)
            logits = model(input_ids)  # (seq_len, batch, vocab_size)
            loss = criterion(logits.view(-1, vocab_size), target_ids.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")


dataloader = DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)
model = SketchTransformer(
    vocab_size=len(tokenizer.vocab), d_model=384, nhead=8, num_layers=6
)

# d_model => model capacity (types of drawing features it can learn)
# nhead => model can attend to more positions in parallel
# num layers => model learns more hierarchical abstractions (patterns, shapes , layouts)

train_model(
    model,
    dataloader,
    vocab_size=len(tokenizer.vocab),
    epochs=100,
    lr=1e-4,
    device=device,
)

c:\Users\matth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Epoch 1/100: 100%|██████████| 5/5 [00:02<00:00,  2.44it/s]


Epoch 1 Loss: 5.0324


Epoch 2/100: 100%|██████████| 5/5 [00:01<00:00,  2.96it/s]


Epoch 2 Loss: 3.5292


Epoch 3/100: 100%|██████████| 5/5 [00:01<00:00,  3.01it/s]


...

Epoch 98 Loss: 1.6276


Epoch 99/100: 100%|██████████| 5/5 [00:01<00:00,  2.90it/s]


Epoch 99 Loss: 1.6174


Epoch 100/100: 100%|██████████| 5/5 [00:01<00:00,  2.89it/s]

Epoch 100 Loss: 1.6209


In [14]:
def top_p_filtering(logits, p=0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = False
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[:, indices_to_remove] = -float("Inf")
    return logits

def top_k_filtering(logits, k):
    if k <= 0:
        return logits
    top_k = min(k, logits.size(-1))
    values, _ = torch.topk(logits, top_k)
    min_values = values[:, -1].unsqueeze(-1)
    logits[logits < min_values] = -float("Inf")
    return logits


def sample_sequence_feat(
    model,
    start_token,
    max_len=200,
    temperature=1.0,
    top_k=60,
    top_p=0.9,
    greedy=False,
    eos_id=None,
    device="cuda",
):
    model.eval()
    tokens = [start_token]
    tokens_tensor = torch.tensor([tokens], device=device)

    for _ in range(max_len - 1):
        with torch.no_grad():
            logits = model(tokens_tensor)
            next_logits = logits[:, -1, :] / temperature

            # top-k / top-p filtering
            next_logits = top_k_filtering(next_logits, top_k)
            next_logits = top_p_filtering(next_logits, top_p)
            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)

        if eos_id is not None and next_token == eos_id:
            break

        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens


start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)
generations_inline = ""

for i in range(5):
    generated = sample_sequence_feat(
        model,
        start_token,
        max_len=200,
        temperature=1.0,
        greedy=False,
        eos_id=eos_token,
        device=device,
    )
    decoded_sketch = tokenizer.decode(generated, stroke_width=0.3)
    
    # print("Generated token sequence:", generated)
    # print("Decoded sketch:", decoded_sketch)
    generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'


from IPython.display import HTML, display
display(HTML(generations_inline))